In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd
import time
import requests
import sys
# import base64, os

In [2]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [C:\Users\Admin\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [3]:
book_name = 'release that witch'
book_name = input()

mages are too op


In [ ]:
try:
    seperator = '-'
    book_name = seperator.join(book_name.split(' '))
    url = f"https://boxnovel.com/novel/{book_name}-boxnovel/"
    browser.visit(url)
    if requests.get(url).status_code == 404:
#         sys.exit()
    time.sleep(1)
except:
    print('Welp me I broke')
#     sys.exit()
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
container = soup.find_all("div", {"class": "summary_content"})
author = container[3].find("div", {"class": "author-content"}).find_all('a')

In [ ]:
with open(f'{book}.txt', 'w',encoding ='utf8') as out:
        chapter = 1
        seperator = ', '
        author_text = seperator.join([author[0].text, author[1].text])
        BookName = ' '.join([word.capitalize() for word in book.split('-')])
        out.write('\t\t\t\t\t\t\t\t\t\t\t\t'+BookName+'\n\n')
        out.write('\t\t\t\t\t\t\t\t\t\t\t'+ Author:'+author_text+'\n\n\n')
        
        try:
            url = f"https://boxnovel.com/novel/{book}/chapter-{chapter}"
            chapter +=1
            browser.visit(url)
            time.sleep(5)
            while True:
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')
                try:
                    container = soup.find_all("div", {"class": "read-container"})
                    text_left = container[0].find_all("div", {"class": "text-left"})
                    chp_title = text_left[0].find_all("p")[0].text
                    pgraphs = text_left[0].find_all("p")
                except:
                    print('swomethwing bwroke mwe')
                    break
                pgraphs.pop(0)
                chapter_text = [p.text for p in pgraphs]

                out.write('-----------------------------------------'+chp_title+'-----------------------------------------\n\n')
                for text in chapter_text:
                    out.write(text+'\n\n')
                try :
                    next_page = browser.driver.find_element_by_class_name('next_page')
                    next_page.click()
                except :
                    break
            except KeyboardInterrupt:
                print ('\nPausing...  (Hit ENTER to continue, type quit to exit.)')
                try:
                    response = input()
                    if response == 'quit':
                        break
                    print ('Resuming...')
                except KeyboardInterrupt:
                    print ('Resuming...')
                    continue
    browser.driver.close()

In [26]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
book = 'Emperor of Solo Play'
# book_name = input()

try:
    seperator = '-'
    book = seperator.join(book.split(' '))
    url = f"https://www.wuxiaworld.co/{book}/"
    browser.visit(url)
#     if requests.get(url).status_code == 404:
#         browser.driver.close()
#         sys.exit()
    time.sleep(10)
except:
    print('Hwelp mwe! I bwroke.')
    try:
        browser.driver.close()
    except:
        print('nothing')
#         sys.exit()
#     sys.exit()
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
container = soup.find_all("div", {"class": "person-info"})
author = container[1].find("div", {"class": "author"}).find_all('span')[1].text
chapter_list = soup.find_all("ul", {"class": "chapter-list"})
chapters = browser.driver.find_elements_by_class_name("chapter-item")

# In[ ]:

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [C:\Users\Admin\.wdm\drivers\chromedriver\win32\85.0.4183.87\chromedriver.exe] found in cache


In [27]:
with open(f'{book}.txt', 'w',encoding ='utf8') as out:
    seperator = ', '
    BookName = ' '.join([word.capitalize() for word in book.split('-')])
    out.write('\t\t\t\t\t\t\t\t\t\t\t\t'+BookName+'\n\n')
    out.write('\t\t\t\t\t\t\t\t\t\t\t Author:'+author+'\n\n\n')

    try:
        # url = f"https://boxnovel.com/novel/{book}/chapter-{chapter}"
        chapters[185].click()
        # browser.visit(url)
        time.sleep(5)
        while True:
            try:
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(5)
                try:
                    container = soup.find_all("div", {"class": "chapter-entity"})
                    title = soup.find_all("h1", {"class": "chapter-title"})[0].text
                    for br in container[0].find_all("br"):
                        br.replace_with("\n")
                    chapter_text =re.sub('((\\t)*(\\n)*[\w ]* https:[/\w\.\- ]*)', '\n', container[0].text)
                except:
                    print('swomethwing bwroke mwe')
                    break
                out.write('-----------------------------------------'+title+'-----------------------------------------\n\n')
                out.write(chapter_text)
                try :
                    next_page = browser.driver.find_element_by_class_name('next')
                    browser.driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_page)
                    next_page.click()
                except :
                    break
            except KeyboardInterrupt:
                print ('\nPausing...  (Hit ENTER to continue, type quit to exit.)')
                try:
                    response = input()
                    if response == 'quit':
                        break
                    print ('Resuming...')
                except KeyboardInterrupt:
                    print ('Resuming...')
                    continue
    except:
        print('Fwalure')
browser.driver.close()